In [3]:
# Load the lightning model from checkpoint

import pytorch_lightning as pl
import torch    
import numpy as np
from torch.utils.data import DataLoader  # Import DataLoader
from src.failure_model import ToxicClassifier
from src.datasets import PromptOnlyDataset, PropDataset
import torch._dynamo
torch._dynamo.config.suppress_errors = True

try:
    torch.multiprocessing.set_start_method('spawn')
except RuntimeError:
    pass

cal_prompts = PromptOnlyDataset("data/rtp_500/split_1_0.5_0.1_0.2_0.2/cal.pkl")

cal_props = PropDataset("data/rtp_500/split_1_0.5_0.1_0.2_0.2/cal.pkl", score_name="toxicity", threshold=0.5)

# cal_prompts.data = cal_prompts.data[-100:]
# cal_props.data = cal_props.data[-100:]

# Examine the model weights in the checkpoint before loading
model_path = "saved/Prop_rtp_500_ModernBERT/lightning_logs/version_0/checkpoints/epoch=4-step=495.ckpt"
model = ToxicClassifier.load_from_checkpoint(model_path)

taus = torch.tensor(np.logspace(-3, 0, 200))
# Get the index closest to 0.1
idx = (taus - 0.1).abs().argmin()
model.set_taus(taus)
model.set_min_p_for_q_tau(1e-20)
# model.set_threshold_p_for_q_tau(1e-4)
model.eval()

trainer = pl.Trainer()

# Create a custom DataLoader for prediction with a batch size of 1500
predict_dataloader = DataLoader(cal_prompts, batch_size=1500, shuffle=False)

# Use the trainer and predict on cal_prompts using the custom DataLoader
pred = trainer.predict(model, dataloaders=predict_dataloader)

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/fre.gilad/miniforge3/envs/vllm4/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/fre.gilad/miniforge3/envs/vllm4/lib/python3.11 ...
You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/fre.gilad/miniforge3/envs/vllm4/lib/pyt

Predicting: |          | 0/? [00:00<?, ?it/s]

In [4]:
# import pickle

# cal_path = "data/rtp_500/split_1_0.5_0.1_0.2_0.2/cal.pkl"

# with open(cal_path, "rb") as f:
#     raw_data = pickle.load(f)

In [6]:
from src.survival_runner import SurvivalRunner
from src.generation.vanilla_model_vllm import VanillaGeneratorVLLM
from src import utils
from src.rating.detoxify import DetoxifyRater

rater_backend = DetoxifyRater(model_type="unbiased", amp=True)


hf_key = utils.api_key_from_file("HF_KEY.txt")
generator_backend = VanillaGeneratorVLLM(
    model_name="meta-llama/Llama-3.2-3B",
    hf_token=hf_key,
    max_output_tokens=50,
    # model_args={"gpu_memory_utilization": 0.95},
)

runner = SurvivalRunner(generator=generator_backend, rater=rater_backend, max_attempts=10)
res = list(runner.generate(prompts=cal_prompts))

INFO: Overhead tokens:  100
INFO: Empty input tokens:  1
INFO: Total sequence tokens:  151


config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

INFO 04-28 22:43:38 [config.py:585] This model supports multiple tasks: {'generate', 'score', 'reward', 'classify', 'embed'}. Defaulting to 'generate'.
INFO 04-28 22:43:38 [config.py:1697] Chunked prefill is enabled with max_num_batched_tokens=151100.


generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

WARNING 04-28 22:43:39 [utils.py:2181] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/getting_started/troubleshooting.html#python-multiprocessing for more information. Reason: CUDA is initialized
INFO 04-28 22:43:44 [__init__.py:239] Automatically detected platform cuda.
INFO 04-28 22:43:46 [core.py:54] Initializing a V1 LLM engine (v0.8.2) with config: model='meta-llama/Llama-3.2-3B', speculative_config=None, tokenizer='meta-llama/Llama-3.2-3B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=251, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgram

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:32<00:32, 32.63s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:41<00:00, 18.75s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:41<00:00, 20.83s/it]



INFO 04-28 22:45:51 [loader.py:447] Loading weights took 41.70 seconds
INFO 04-28 22:45:52 [gpu_model_runner.py:1186] Model loading took 6.0169 GB and 124.386554 seconds
INFO 04-28 22:46:00 [backends.py:415] Using cache directory: /home/fre.gilad/.cache/vllm/torch_compile_cache/d7405ee669/rank_0_0 for vLLM's torch.compile
INFO 04-28 22:46:00 [backends.py:425] Dynamo bytecode transform time: 8.55 s
INFO 04-28 22:46:04 [backends.py:132] Cache the graph of shape None for later use
INFO 04-28 22:46:29 [backends.py:144] Compiling a graph for general shape takes 28.42 s
INFO 04-28 22:46:54 [monitor.py:33] torch.compile takes 36.96 s in total
INFO 04-28 22:46:55 [kv_cache_utils.py:566] GPU KV cache size: 41,888 tokens
INFO 04-28 22:46:55 [kv_cache_utils.py:569] Maximum concurrency for 251 tokens per request: 166.88x
INFO 04-28 22:47:13 [gpu_model_runner.py:1534] Graph capturing finished in 17 secs, took 1.41 GiB
INFO 04-28 22:47:13 [core.py:151] init engine (profile, create kv cache, warmup m

Processing Prompts:   0%|          | 0/19888 [00:00<?, ?it/s]

In [7]:
res

[]

In [ ]:
raw_data[0].ratings

In [ ]:
raw_data[0].ratings

In [ ]:
# concatanate all rating arrays
from tqdm.auto import tqdm

all_ratings = []

for res in tqdm(raw_data):
    scores = [rt.scores["toxicity"] for rt in res.ratings]
    all_ratings.extend(scores)

In [ ]:
# plot histogram of all ratings
# show number of ratings in each bin

import matplotlib.pyplot as plt

counts, edges, bars = plt.hist(all_ratings, bins=4)
plt.bar_label(bars)
plt.xlabel("Rating")
plt.ylabel("Count")
plt.title("Histogram of Individual Ratings")
plt.show()

In [ ]:
from src.datasets import PropDataset

cal_props = PropDataset("data/rtp_500/split_1_0.5_0.1_0.2_0.2/cal.pkl", score_name="toxicity", threshold=0.5)

In [ ]:
avg_scores = [y[0] for (x, y) in tqdm(cal_props)]

In [ ]:
counts, edges, bars = plt.hist(avg_scores, bins=4)
plt.bar_label(bars)
plt.xlabel("Rating")
plt.ylabel("Count")
plt.title("Histogram of Average Ratings")
plt.show()

In [ ]:
# Adapted github.com/unitaryai/detoxify
import transformers

import pytorch_lightning as pl
from src.loss import survival_loss, prop_loss
import torch
from torch.nn import functional as F
import torchmetrics
from scipy.stats import geom

In [ ]:
DOWNLOAD_URL = "https://github.com/unitaryai/detoxify/releases/download/"
MODEL_URLS = {
    "original": DOWNLOAD_URL + "v0.1-alpha/toxic_original-c1212f89.ckpt",
    "unbiased": DOWNLOAD_URL + "v0.3-alpha/toxic_debiased-c7548aa0.ckpt",
    "multilingual": DOWNLOAD_URL + "v0.4-alpha/multilingual_debiased-0b549669.ckpt",
    "original-small": DOWNLOAD_URL + "v0.1.2/original-albert-0e1d6498.ckpt",
    "unbiased-small": DOWNLOAD_URL + "v0.1.2/unbiased-albert-c8519128.ckpt",
}

device = "cpu"
model_type = "original"
checkpoint_path = MODEL_URLS[model_type]
loaded = torch.hub.load_state_dict_from_url(checkpoint_path, map_location=device)
state_dict = loaded["state_dict"]

In [ ]:
from src.failure_model import ToxicClassifier
import json


config_path = "/home/fre.gilad/source/llm-survival/configs/Prop_RTP_500_BERT.json"
config = json.load(open(config_path))

model = ToxicClassifier(
    config=config,
)

In [ ]:
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
from src import utils

hf_key = utils.api_key_from_file("HF_KEY.txt")

In [ ]:
from torch import Tensor


def weighted_mse(input: Tensor, target: Tensor, weight: Tensor) -> Tensor:
    """
    Computes the weighted mean squared error loss.
    """
    return torch.mean(weight * (input - target) ** 2)

In [ ]:
from torch import Tensor
from typing import Literal
import math


class ToxicClassifierProb(pl.LightningModule):
    def __init__(
        self,
        model_name: str,
        loss_type: Literal["bce", "scaled_bce", "log_mse", "quantile_mse"],
        hf_token: str,
        optim_params=None,
        max_survival_time: float = 1500,
    ):
        super().__init__()

        assert loss_type in ["bce", "scaled_bce", "log_mse", "quantile_mse"], f"Invalid loss type: {loss_type}"

        self.save_hyperparameters()

        if optim_params is None:
            optim_params = {}

        self.optim_params = optim_params
        self.max_survival_time = max_survival_time
        self.loss_type = loss_type

        self.min_prob = 1 / self.max_survival_time
        self.max_prob = 1 - 1 / self.max_survival_time

        device = "cuda" if torch.cuda.is_available() else "cpu"

        self.tokenizer = transformers.AutoTokenizer.from_pretrained(
            model_name,
            device_map=device,
            token=hf_token,
        )

        self.model = transformers.AutoModelForSequenceClassification.from_pretrained(
            model_name,
            num_labels=1,
            token=hf_token,
        )

        self.to(device)
        self.train()

        if self.loss_type == "bce":
            self.loss_fn = self.bce_loss
        elif self.loss_type == "scaled_bce":
            self.loss_fn = self.scaled_bce_loss
        elif self.loss_type == "log_mse":
            self.loss_fn = self.log_mse_loss
        elif self.loss_type == "quantile_mse":
            self.loss_fn = self.quantile_mse_loss

    def to_device(self, item):
        if isinstance(item, torch.Tensor):
            return item.to(self.device)
        elif isinstance(item, dict):
            return {k: self.to_device(v) for k, v in item.items()}
        elif isinstance(item, (list, tuple)):
            return [self.to_device(i) for i in item]

        return item

    def forward(self, prompts) -> torch.Tensor:
        """
        Predict probability directly
        """
        inputs = self.tokenizer(prompts, return_tensors="pt", truncation=True, padding=True)
        inputs = inputs.to(self.device)
        logits = self.model(**inputs)[0]

        # predict p directly
        return torch.sigmoid(logits).flatten()

    def bce_loss(self, pred_probs: Tensor, meta: tuple[Tensor, ...]) -> Tensor:
        """
        Computes the binary cross entropy loss between predicted and true survival probs.
        """
        true_probs, _ = meta
        true_probs = true_probs.to(pred_probs.dtype)
        true_probs = torch.clamp(true_probs, min=self.min_prob, max=self.max_prob)
        pred_probs = torch.clamp(pred_probs, min=self.min_prob, max=self.max_prob)
        return F.binary_cross_entropy(pred_probs, true_probs)

    def scaled_bce_loss(self, pred_probs: Tensor, meta: tuple[Tensor, ...]) -> Tensor:
        true_probs, _ = meta
        true_probs = true_probs.to(pred_probs.dtype)
        true_probs = torch.clamp(true_probs, min=self.min_prob, max=self.max_prob)
        pred_probs = torch.clamp(pred_probs, min=self.min_prob, max=self.max_prob)
        return F.binary_cross_entropy(pred_probs, true_probs, weight=torch.sqrt(1 / true_probs))

    def log_mse_loss(self, pred_probs: Tensor, meta: tuple[Tensor, ...]) -> Tensor:
        true_probs, _ = meta
        true_probs = true_probs.to(pred_probs.dtype)
        true_probs = torch.clamp(true_probs, min=self.min_prob, max=self.max_prob)
        pred_probs = torch.clamp(pred_probs, min=self.min_prob, max=self.max_prob)

        true_logs = torch.log(true_probs / (1 - true_probs))
        pred_logs = torch.log(pred_probs / (1 - pred_probs))
        return F.mse_loss(pred_logs, true_logs)

    def quantile_mse_loss(self, pred_probs: Tensor, meta: tuple[Tensor, ...], quantile: float = 0.5) -> Tensor:
        true_probs, _ = meta
        true_probs = true_probs.to(pred_probs.dtype)
        true_probs = torch.clamp(true_probs, min=self.min_prob, max=self.max_prob)
        pred_probs = torch.clamp(pred_probs, min=self.min_prob, max=self.max_prob)

        true_quant = math.log(1 - quantile) / torch.log(1 - true_probs)
        pred_quant = math.log(1 - quantile) / torch.log(1 - pred_probs)
        return F.mse_loss(pred_quant, true_quant)

    def training_step(self, batch, batch_idx) -> torch.Tensor:
        prompts, meta = batch
        pred_probs = self.forward(prompts)
        meta = self.to_device(meta)
        loss = self.loss_fn(pred_probs, meta)
        return loss

    def validation_step(self, batch, batch_idx) -> None:
        prompts, meta = batch
        pred_probs = self.forward(prompts)

        meta = self.to_device(meta)
        bce_loss = self.bce_loss(pred_probs, meta)
        scaled_bce_loss = self.scaled_bce_loss(pred_probs, meta)
        log_mse_loss = self.log_mse_loss(pred_probs, meta)
        quant_05_loss = self.quantile_mse_loss(pred_probs, meta, quantile=0.5)

        self.log("val/bce_loss", bce_loss, prog_bar=True)
        self.log("val/scaled_bce_loss", scaled_bce_loss, prog_bar=True)
        self.log("val/log_mse_loss", log_mse_loss, prog_bar=True)
        self.log("val/quant_05_loss", quant_05_loss, prog_bar=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), **self.optim_params)

In [ ]:
from src.datasets import PropDataset
from pytorch_lightning.callbacks import ModelCheckpoint


model_name = "answerdotai/ModernBERT-base"

for loss_type in ["bce", "scaled_bce", "log_mse", "quantile_mse"]:

    model = ToxicClassifierProb(
        model_name=model_name,
        hf_token=hf_key,
        loss_type=loss_type,
        optim_params={"lr": 2e-5},
        max_survival_time=500,
    )

    train_path = "data/rtp_500/split_1_0.5_0.1_0.2_0.2/train.pkl"
    valid_path = "data/rtp_500/split_1_0.5_0.1_0.2_0.2/val.pkl"

    ds_train = PropDataset(
        train_path,
        score_name="toxicity",
        threshold=0.5,
    )

    ds_valid = PropDataset(
        valid_path,
        score_name="toxicity",
        threshold=0.5,
    )

    batch_size = 128
    num_workers = 0

    dl_train = torch.utils.data.DataLoader(
        ds_train,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
    )

    dl_valid = torch.utils.data.DataLoader(
        ds_valid,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
    )

    max_epochs = 20
    accumulate_grad_batches = 1

    checkpoint_callback = ModelCheckpoint(
        save_top_k=1,
        verbose=True,
        monitor="val/quant_05_loss",
        mode="min",
    )

    trainer = pl.Trainer(
        devices="auto",
        max_epochs=max_epochs,
        precision=32,
        logger=True,
        enable_checkpointing=True,
        accumulate_grad_batches=accumulate_grad_batches,
        callbacks=[checkpoint_callback],
    )

    trainer.fit(
        model=model,
        train_dataloaders=dl_train,
        val_dataloaders=dl_valid,
    )